In [1]:
# 配置环境依赖
ROOT = '/kaggle/working'

!pip install triton tqdm timm fairscale transformers accelerate diffusers albumentations einops safetensors
!pip install bitsandbytes==0.35.0

!git clone https://github.com/kohya-ss/sd-scripts.git

!cp -r /kaggle/input/train-images/image/100_Neilina {ROOT}/image
!ln -s /kaggle/input/chilloutmix-safetensors-models/chilloutmix-Ni-pruned-fp16.safetensors

%cd {ROOT}/sd-scripts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.2 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307224 sha256=db1e75495d5c9a9b8ce82d9d00ff9f06e29fe5310316a8bd3c9fe81f61f777aa
  Stored in directory: /root/.cache/pip/wheels/0b/8c/fa/a9e102632bcb86e919561cf25ca1e0dd2ec67476f3a5544653
Successfully built fairscale
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 15.6 MB/s eta 0:00:00
Cl

In [2]:
# 将sd-scripts/library作为库复制到python的site-packages中，供调用
!touch '{ROOT}/sd-scripts/library/__init__.py'
!ln -s '{ROOT}/sd-scripts/library/' '/opt/conda/lib/python3.7/site-packages/library'

# 打标    
!python './finetune/make_captions.py' \
        '{ROOT}/100_Neilina' \
        --batch_size 8

Current Working Directory is:  /kaggle/working/sd-scripts
load images from /kaggle/working/100_Neilina
found 0 images.
loading BLIP caption: https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
100%|███████████████████████████████████████| 1.66G/1.66G [00:09<00:00, 194MB/s]
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
BLIP loaded
0it [00:00, ?it/s]
done!


In [3]:
# 训练参数
MODEL_PATH = f"{ROOT}/chilloutmix-Ni-pruned-fp16.safetensors" # checkpoint 路径
TRAIN_DATA_DIR = f"{ROOT}/image" # 训练数据集路径
OUTPUT_DIR = f"{ROOT}/output" # 输出模型路径
OUTPUT_NAME = "neilina" # 输出模型名称

RESOLUTION = "512,512" # image resolution w,h. 图片分辨率，宽,高。支持非正方形，但必须是 64 倍数。
LR = "1e-4" # learning rate 学习率
MAX_TRAIN_EPOCHS =10 # 最大训练 epoch
SAVE_EVERY_N_EPOCHS = 4 # 每 N 个 epoch 保存一次
NETWORK_DIM = 32 # 常用 4~128，不是越大越好
NEWORK_ALPHA= 32 # 常用与 network_dim 相同的值或者采用较小的值，如 network_dim的一半 防止下溢。默认值为 1，使用较小的 alpha 需要提升学习率。

In [4]:
# 训练
!python train_network.py \
  --network_dim={NETWORK_DIM} \
  --network_alpha={NEWORK_ALPHA} \
  --pretrained_model_name_or_path={MODEL_PATH} \
  --train_data_dir={TRAIN_DATA_DIR} \
  --output_dir={OUTPUT_DIR} \
  --output_name={OUTPUT_NAME} \
  --resolution={RESOLUTION} \
  --learning_rate={LR} \
  --max_train_epochs={MAX_TRAIN_EPOCHS} \
  --save_every_n_epochs={SAVE_EVERY_N_EPOCHS} \
  --train_batch_size=1 \
  --mixed_precision=fp16 \
  --save_precision=fp16 \
  --seed=42 \
  --clip_skip=2 \
  --prior_loss_weight=1 \
  --network_module=networks.lora \
  --shuffle_caption

prepare tokenizer
Use DreamBooth method.
prepare train images.
0 train images with repeating.
loading image sizes.
0it [00:00, ?it/s]
prepare dataset
No data found. Please verify arguments (train_data_dir must be the parent of folders with images) / 画像がありません。引数指定を確認してください（train_data_dirには画像があるフォルダではなく、画像があるフォルダの親フォルダを指定する必要があります）
